In [2]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [5]:
df = pd.read_csv('data/all/meteo/sampled_df_10000.csv')

In [6]:
df.drop(['date', 'geometry', 'TIPOBOSQUE', 'UsoIFN', 'n_CLAMFE', 'UsoMFE'], axis=1, inplace=True)
df

,altitud,pendiente,orientacion,is_fire,n_CLAIFN,t_max,u,v,specific_humidity,relative_humidity
0,226.43176,1.512323,287.237460,False,200,302.235107,-0.482693,-1.057308,0.011643,51.497208
1,591.98520,37.829388,174.613740,True,141,291.587646,5.145885,-2.061542,0.002844,29.498014
2,302.96150,57.214653,76.929750,True,111,306.062988,0.700964,0.509414,0.006809,24.005035
3,365.60150,36.542430,112.896065,False,141,298.082031,-2.383409,-1.082997,0.013121,67.682327
4,216.34225,48.399876,129.588600,True,140,305.471191,4.591745,-0.290295,0.004589,14.796286
...,...,...,...,...,...,...,...,...,...,...
9995,321.46176,58.597137,240.593480,False,121,300.824463,-1.734298,0.723990,0.010968,48.677845
9996,998.84950,23.794138,231.235290,True,171,302.501221,-1.233615,-0.148222,0.009740,39.612019
9997,79.08550,39.971180,339.268550,False,300,280.333252,-1.505415,-3.901253,0.003711,66.535889
9998,288.20276,34.997368,306.008120,False,111,291.637451,1.306460,0.193386,0.004443,35.163040


In [7]:
# Select quantitatives variables
cuant_var = df[["altitud", "pendiente", "orientacion", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = cuant_var.mean()
mean.to_csv(basePath + '/means.csv')
std = cuant_var.std()
std.to_csv(basePath + '/stds.csv')
normalized_df = (cuant_var-cuant_var.mean())/cuant_var.std()
# Add soil use binarizaded
normalized_df = normalized_df.join(pd.get_dummies(df["n_CLAIFN"].astype("category")))

In [8]:
# Split dataset
y = df["is_fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [9]:
xTest

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity,111,112,...,141,142,150,161,171,172,200,300,400,500
5669,0.791761,0.281929,-0.155047,0.570548,-0.461864,0.027699,0.485999,0.015338,1,0,...,0,0,0,0,0,0,0,0,0,0
8800,1.080497,-0.713308,-1.345743,0.026523,-0.910445,-0.049421,1.275303,1.257824,0,0,...,1,0,0,0,0,0,0,0,0,0
3205,0.769821,0.982115,-1.592882,-0.145930,-1.630387,-1.061468,2.811011,1.934182,0,0,...,1,0,0,0,0,0,0,0,0,0
8731,0.395582,-0.511498,0.443695,1.406692,0.616303,0.241257,-0.415831,-1.431236,1,0,...,0,0,0,0,0,0,0,0,0,0
6412,1.695439,1.466722,0.173093,-0.481614,-0.994136,-0.547441,0.463707,0.825513,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,-1.362713,-0.692165,-1.648495,-1.257524,-1.178980,-0.579903,-1.169310,-0.317053,0,0,...,0,0,0,0,0,0,0,1,0,0
142,0.931766,-0.062998,1.105972,-1.612311,1.982156,-2.971841,-1.622293,-0.551655,0,0,...,0,0,0,0,0,0,0,0,0,0
275,-0.889621,1.199585,-1.482392,-1.238219,-1.543729,-1.785804,0.126060,1.526523,0,0,...,0,0,0,0,0,0,0,1,0,0
8265,0.497148,0.318305,-0.947187,0.630667,-0.403922,0.123957,0.285282,-0.151447,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Create function returning a model
def design_model (dropout_rate, weight_constraint):

    model = keras.Sequential()

    model.add(keras.layers.Dense(64, input_shape=(26,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

model = KerasClassifier(model=design_model, verbose=0)

In [11]:
# Define the search parameters
epochs = [10,20,50,100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [ 0.2, 0.3, 0.4, 0.5]

In [12]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint, optimizer=optimizer, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)

In [13]:
grid_result = grid.fit(xTrain, yTrain)

Fitting 3 folds for each of 560 candidates, totalling 1680 fits


In [15]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"{mean} {stdev} with: {param}")

Best:0.9348733415074802 using {'epochs': 100, 'model__dropout_rate': 0.2, 'model__weight_constraint': 3.0, 'optimizer': 'Nadam'}
0.9132489346558613 0.007576749820481045 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'SGD'}
0.9168746536090341 0.004436142947007936 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'RMSprop'}
0.9128744503336182 0.0032021947538267595 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adagrad'}
0.9166239816422047 0.007463477130135467 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adadelta'}
0.9132494503453384 0.004472270329647967 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adam'}
0.9154987472027362 0.008127965384990105 with: {'epochs': 10, 'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0, 'optimizer': 'Adamax'}
0

In [18]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

Best:0.9348733415074802 using {'epochs': 100, 'model__dropout_rate': 0.2, 'model__weight_constraint': 3.0, 'optimizer': 'Nadam'}


In [19]:
##### Design final model with best parameters
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_shape=(26,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(3)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))
    
model.add(keras.layers.Dense(2, activation='softmax'))

In [20]:
# Define the accuracy metrics and parameters
model.compile(optimizer="Nadam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=100)

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("\nConfusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Epoch 1/100
250/250 [==============================] - 5s 4ms/step - loss: 0.3277 - accuracy: 0.8684
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2566 - accuracy: 0.9045
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2450 - accuracy: 0.9060
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2364 - accuracy: 0.9111
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2290 - accuracy: 0.9137
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2291 - accuracy: 0.9130
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2155 - accuracy: 0.9221
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2140 - accuracy: 0.9202
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2086 - accuracy: 0.9200
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.2107 - accura

In [21]:
# Export model
model.save('data/nn/nn_fire_risk_all_26_optimized.h5')

In [22]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.9
    Uninstalling flax-0.6.9:
      Successfully uninstalled flax-0.6.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
xar

In [ ]:
# Load model
model = keras.models.load_model('data/nn/nn_fire_risk_all_26_optimized.h5')

In [23]:
import tensorflowjs as tfjs

# Convert the Keras model to TensorFlow.js format
tfjs.converters.save_keras_model(model, 'data/nn/nn_fire_risk_all_26_optimized_js')